In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[link to original NB](https://www.kaggle.com/alexandraamidon/simple-hungry-geese-baseline-heuristic-agent/notebook)

# Play default agents
Just to get a sense of how the game works, play two types of default agents against each other: a greedy agent and a random agent.

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import evaluate, make
env = make("hungry_geese")
env.reset()
env.run([random_agent, random_agent, random_agent, random_agent])
env.render(mode="ipython", width=800, height=700)

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import evaluate, make
env = make("hungry_geese")
env.reset()
env.run([greedy_agent, greedy_agent, greedy_agent, greedy_agent])
env.render(mode="ipython", width=800, height=700)

# Simple Baseline: Improve Greedy Agent with 1-step lookahead

As a start, I will make a simple improvement to the greedy agent: have it look 1 step ahead. 

1. Don't get stuck. In the game matching greedy agents against eachother, agents often lost when they got "stuck" and couldn't avoid moving into an adversary or a body. We can look ahead 1 step and avoid doing that, if possible.

2. Be adversarial. If agent can move to make another agent "stuck", do so.

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, adjacent_positions, translate, min_distance
import numpy as np
from random import choice


class StrategicGreedyAgent:
    def __init__(self, configuration: Configuration):
        self.configuration = configuration
        self.last_action = None

    def __call__(self, observation: Observation):
        rows, columns = self.configuration.rows, self.configuration.columns

        food = observation.food
        geese = observation.geese
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]

        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent in opponents
            for opponent_head in [opponent[0]]
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        # Don't move into any bodies
        bodies = {position for goose in geese for position in goose}
        
        position = geese[observation.index][0] 
        
        # Don't get stuck
        stuck_actions = []
        for action in Action:
            new_position = translate(position, action, columns, rows)
            next_actions = [next_action for next_action in Action if next_action != action.opposite()]
            num_moves = 0
            for next_action in next_actions:
                next_position = translate(new_position, next_action, columns, rows)
                if next_position not in head_adjacent_positions and next_position not in bodies:
                    num_moves += 1
            if num_moves <= 1:
                stuck_actions.append(next_action)
        # safety layer - make sure goose can do something!
        if len(stuck_actions) == 4:
            stuck_actions = []
        
        # Move to the closest food
        actions = {
            action: min_distance(new_position, food, columns)
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in head_adjacent_positions and
                new_position not in bodies and
                action not in stuck_actions and 
                (self.last_action is None or action != self.last_action.opposite())
            )
        }
        
        action = min(actions, key=actions.get) if any(actions) else choice([action for action in Action])
        self.last_action = action
        return action.name


cached_greedy_agents = {}


def agent(obs, config):
    index = obs["index"]
    if index not in cached_greedy_agents:
        cached_greedy_agents[index] = StrategicGreedyAgent(Configuration(config))
    return cached_greedy_agents[index](Observation(obs))



In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import evaluate, make
env = make("hungry_geese")
env.reset()
env.run(['submission.py', 'submission.py', 'submission.py', 'submission.py'])
env.render(mode="ipython", width=500, height=400)